<a href="https://colab.research.google.com/github/elsebasantiago/Algoritmos_optimizacion_VIU/blob/main/Algoritmos_AG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3
# Sebastián Santiago
Link repositorio github
https://github.com/elsebasantiago/Algoritmos_optimizacion_VIU

#Carga de librerias

In [2]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


#Carga de los datos del problema

In [3]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



# Nueva sección

In [4]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [5]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 10)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

92

#Funcionas basicas


In [6]:

#Funciones basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion +=  [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)



#BUSQUEDA ALEATORIA

In [10]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 15000)

Mejor solución: [0, 3, 7, 31, 33, 36, 20, 26, 16, 19, 25, 41, 18, 4, 29, 30, 32, 37, 15, 8, 10, 21, 38, 2, 35, 17, 6, 5, 22, 34, 24, 23, 28, 9, 13, 12, 11, 39, 40, 27, 1, 14]
Distancia     : 3544


#BUSQUEDA LOCAL

In [11]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)-1):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3544
Distancia Mejor Solucion Local: 3387


In [9]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  print (f'distancia inicial = {mejor_distancia}')

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

distancia inicial = 4826
En la iteracion  33 , la mejor solución encontrada es: [0, 1, 6, 18, 12, 11, 25, 10, 2, 32, 31, 35, 36, 37, 15, 16, 14, 21, 24, 40, 8, 26, 5, 4, 3, 27, 28, 39, 22, 38, 30, 29, 9, 23, 41, 13, 19, 7, 34, 33, 20, 17]
Distancia     : 2041


In [20]:
#Busqueda Local con multi arranque
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  total_iter = 0
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  mejor_distancia_vecina = mejor_distancia
  print (f'distancia inicial = {mejor_distancia}')

  while(True):
    iteracion=0             #Un contador para saber las iteraciones que hacemos
    next_vecino = True
    while(next_vecino):
      iteracion +=1         #Incrementamos el contador

      #Obtenemos la mejor vecina ...
      vecina = genera_vecina(solucion_referencia)

      #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
      distancia_vecina = distancia_total(vecina, problem)

      #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
      if distancia_vecina < mejor_distancia_vecina:
        mejor_distancia_vecina = distancia_vecina

      else:
        #print("En la iteracion ", iteracion, ", la mejor solución vecina encontrada hasta el momento es:" , vecina)
        print(f'En la iteracion {iteracion} Distancia     : {distancia_vecina}')
        next_vecino = False

      solucion_referencia = vecina

    if mejor_distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = mejor_distancia_vecina
      print("En el arranque ", total_iter, ", la mejor solución encontrada hasta el momento es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)

    solucion_referencia = crear_solucion(Nodos)
    distancia_vecina = distancia_total(solucion_referencia, problem)
    mejor_distancia_vecina = distancia_vecina
    total_iter +=1

    if  total_iter == 30:
      print("La mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion


sol = busqueda_local(problem )

distancia inicial = 4870
En la iteracion 29 Distancia     : 1891
En el arranque  0 , la mejor solución encontrada hasta el momento es: [0, 1, 7, 31, 32, 27, 2, 18, 11, 12, 28, 30, 38, 22, 39, 29, 26, 5, 13, 19, 34, 33, 20, 35, 36, 17, 37, 15, 16, 14, 6, 3, 4, 10, 41, 23, 40, 24, 21, 9, 8, 25]
Distancia     : 1890
En la iteracion 28 Distancia     : 1577
En el arranque  1 , la mejor solución encontrada hasta el momento es: [0, 27, 28, 29, 22, 38, 30, 32, 7, 15, 16, 14, 1, 6, 4, 3, 2, 10, 41, 23, 40, 24, 21, 39, 9, 8, 25, 11, 12, 18, 26, 5, 13, 19, 37, 17, 31, 36, 35, 33, 34, 20]
Distancia     : 1575
En la iteracion 33 Distancia     : 1729
En la iteracion 43 Distancia     : 1847
En la iteracion 30 Distancia     : 1885
En la iteracion 28 Distancia     : 1814
En la iteracion 36 Distancia     : 1827
En la iteracion 32 Distancia     : 1859
En la iteracion 26 Distancia     : 2037
En la iteracion 32 Distancia     : 1817
En la iteracion 33 Distancia     : 1787
En la iteracion 41 Distancia     : 